In [1]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json

env = "production"
#s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-03-14/hr=16/part-00000-76298f84-5414-49c5-88f6-a981ff06d94d-c000.csv .

s3_bucket = "s3://mist-aggregated-stats-{env}/aggregated-stats/".format(env=env)

date_day = "2020-03-19"
hr = '20'

rrm_local_path = "top_1_time_epoch_by_site_ap_ap2_band/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)

rrm_local_path = s3_bucket + rrm_local_path
print(rrm_local_path)



# user_org_id = "38a18d4d-1623-4985-86d0-1bb06e5e2a48"  # UPS
# user_site_USNYELE = '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1'  #UPS USNYELE

# # user_org_id = '38a18d4d-1623-4985-86d0-1bb06e5e2a48'  # UPS
# user_site_USFLLON = 'c23667f3-47e9-44aa-9761-de49d34ed9f9' # USFLLON


s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-03-19/hr=20/*.csv


In [2]:
rdd_rrm_local = spark.sparkContext.textFile(rrm_local_path)
rdd_rrm_local.take(3)

['site,ap,ap2,band,time_epoch,rssi',
 '0009dab3-7a6b-4ace-85ff-e6dc20768bcb,5c5b358e297c,"",5,1584651493,-50.0',
 '000cb16d-91d5-42e2-9c1f-a0319a8116bd,5c5b35506699,5c5b3550629d,5,1584651407,-80.0']

In [3]:
# rdd_rrm_local.map(lambda line: (line.split(',')[0], line.split(',')[1])).collect()

In [4]:

df_scan = spark.read.format("csv")\
   .option("header", "true")\
   .option("inferSchema", "true")\
   .load(rrm_local_path)
df_scan.createOrReplaceTempView("scan_data")
df_Schema = df_scan.schema

In [5]:
df_Schema

StructType(List(StructField(site,StringType,true),StructField(ap,StringType,true),StructField(ap2,StringType,true),StructField(band,IntegerType,true),StructField(time_epoch,IntegerType,true),StructField(rssi,DoubleType,true)))

In [6]:
df_scan.show()

+--------------------+------------+------------+----+----------+-----+
|                site|          ap|         ap2|band|time_epoch| rssi|
+--------------------+------------+------------+----+----------+-----+
|0009dab3-7a6b-4ac...|5c5b358e297c|        null|   5|1584651493|-50.0|
|000cb16d-91d5-42e...|5c5b35506699|5c5b3550629d|   5|1584651407|-80.0|
|00193f17-4e86-476...|5c5b357eeb7d|5c5b35aed61f|   5|1584651461|-78.0|
|00193f17-4e86-476...|5c5b35aed7e6|5c5b35aed818|  24|1584651462|-59.0|
|00193f17-4e86-476...|5c5b35aed872|5c5b35aed7e6|  24|1584651465|-66.0|
|00193f17-4e86-476...|5c5b35aed87c|5c5b35aed877|  24|1584651466|-75.0|
|00226d33-cd80-4b1...|5c5b35501f9f|5c5b35502003|  24|1584651085|-64.0|
|00252227-5f8f-4e6...|5c5b352e8f25|        null|  24|1584651485|-61.0|
|00272816-8dc2-4f1...|5c5b35ae6dc9|5c5b357f32c7|  24|1584651463|-72.0|
|00458c29-d67b-4cf...|5c5b353e5573|5c5b353e5555|  24|1584651478|-61.0|
|00458c29-d67b-4cf...|5c5b353e565e|5c5b353e579e|   5|1584651499|-67.0|
|00458

In [7]:
df_scan.describe().show()

+-------+--------------------+------------+------------+-----------------+--------------------+------------------+
|summary|                site|          ap|         ap2|             band|          time_epoch|              rssi|
+-------+--------------------+------------+------------+-----------------+--------------------+------------------+
|  count|             2698488|     2698488|     2472892|          2698488|             2698488|           2698488|
|   mean|                null|        null|        null|16.10877943500212|1.5846512685131106E9|-66.59862078319415|
| stddev|                null|        null|        null|9.362791315364358|   630.5341625638516| 8.445860009209433|
|    min|0009dab3-7a6b-4ac...|5c5b350e011c|5c5b350e0090|                5|          1584648000|             -96.0|
|    max|fffb6917-ee0e-45c...|d420b080bb76|d420b080bb76|               24|          1584651598|             -11.0|
+-------+--------------------+------------+------------+-----------------+------

In [8]:
df_scan.count()

2698488

In [9]:
site_id = "d1ee1d22-4b55-4c97-97c4-9d757144f45b"

In [10]:
df_scan_site = df_scan.filter(col('site')==site_id)

In [11]:
df_scan_site.count()

184

In [12]:
df_scan_site.show()

+--------------------+------------+------------+----+----------+-----+
|                site|          ap|         ap2|band|time_epoch| rssi|
+--------------------+------------+------------+----+----------+-----+
|d1ee1d22-4b55-4c9...|5c5b3552b796|5c5b3552b96c|  24|1584651471|-63.0|
|d1ee1d22-4b55-4c9...|5c5b3552b94e|5c5b3552b660|   5|1584651376|-63.0|
|d1ee1d22-4b55-4c9...|5c5b3552b7cd|5c5b3552b796|  24|1584651396|-53.0|
|d1ee1d22-4b55-4c9...|5c5b3552b7cd|5c5b3552bb6f|  24|1584651396|-57.0|
|d1ee1d22-4b55-4c9...|5c5b3552b7cd|5c5b3552b5e3|   5|1584651396|-66.0|
|d1ee1d22-4b55-4c9...|5c5b3552b96c|5c5b3552b796|   5|1584651357|-68.0|
|d1ee1d22-4b55-4c9...|5c5b3552b70a|5c5b3552b7cd|  24|1584651450|-66.0|
|d1ee1d22-4b55-4c9...|5c5b3552b714|5c5b3552b796|  24|1584651358|-56.0|
|d1ee1d22-4b55-4c9...|5c5b3552ba70|5c5b3552b5e3|  24|1584651347|-55.0|
|d1ee1d22-4b55-4c9...|5c5b3552b94e|5c5b3552b5e3|  24|1584651376|-65.0|
|d1ee1d22-4b55-4c9...|5c5b3552ba70|5c5b3552b660|   5|1584651347|-61.0|
|d1ee1

# GraphFrames
The following command creates your first GraphFrame. The GraphFrame accepts two DataFrames as inputs—vertices and edges. GraphFrames like to have a naming convention in the column name, which you need to follow. Those rules are defined as follows:
* A DataFrame that represents vertices should contain a column named id. Here, personsDf contains a column name "id".

* A DataFrame that represents edges should contain columns named src and dst. Here, reationshipDf contains the columns "src" and "dst".

In [13]:
vertices= df_scan_site.selectExpr("ap as id").distinct()

In [14]:
edges=df_scan_site.selectExpr("ap as src", "ap2 as dst", "rssi as weight")

In [15]:
from graphframes import *
g = GraphFrame(vertices, edges)

In [16]:
g.vertices.show()
g.edges.show()
## Check the number of edges of each vertex
g.degrees.show()

+------------+
|          id|
+------------+
|5c5b3552b796|
|5c5b3552ba70|
|d420b000a866|
|5c5b3552b96c|
|5c5b3552bb6f|
|5c5b3552b70a|
|5c5b3552b714|
|5c5b3552b94e|
|5c5b3552b5e3|
|5c5b3552b7cd|
|5c5b3552b660|
+------------+

+------------+------------+------+
|         src|         dst|weight|
+------------+------------+------+
|5c5b3552b796|5c5b3552b96c| -63.0|
|5c5b3552b94e|5c5b3552b660| -63.0|
|5c5b3552b7cd|5c5b3552b796| -53.0|
|5c5b3552b7cd|5c5b3552bb6f| -57.0|
|5c5b3552b7cd|5c5b3552b5e3| -66.0|
|5c5b3552b96c|5c5b3552b796| -68.0|
|5c5b3552b70a|5c5b3552b7cd| -66.0|
|5c5b3552b714|5c5b3552b796| -56.0|
|5c5b3552ba70|5c5b3552b5e3| -55.0|
|5c5b3552b94e|5c5b3552b5e3| -65.0|
|5c5b3552ba70|5c5b3552b660| -61.0|
|5c5b3552b96c|5c5b3552bb6f| -68.0|
|d420b000a866|5c5b3552ba70| -80.0|
|5c5b3552b5e3|5c5b3552b660| -56.0|
|5c5b3552b96c|5c5b3552b7cd| -54.0|
|5c5b3552b5e3|5c5b3552b94e| -67.0|
|5c5b3552b94e|5c5b3552b660| -56.0|
|5c5b3552bb6f|        null| -70.0|
|5c5b3552b796|5c5b3552b70a| -60.0|
|5c5

In [17]:
display(g)

GraphFrame(v:[id: string], e:[src: string, dst: string ... 1 more field])

In [18]:
# The following code gets the number of incoming links to Andrew. This is obtained by calling the inDegrees method.

g.degrees.filter("id = '5c5b3552b5e3'").show()

+------------+------+
|          id|degree|
+------------+------+
|5c5b3552b5e3|    36|
+------------+------+



In [19]:
# The following code shows how to get the number of links coming out from Andrew using the outDegrees method .

g.outDegrees.filter("id = '5c5b3552b5e3'").show()


+------------+---------+
|          id|outDegree|
+------------+---------+
|5c5b3552b5e3|       18|
+------------+---------+



In [20]:
ranks = g.pageRank(resetProbability=0.10, maxIter=5)
# display(ranks.vertices.select("id","pagerank").orderBy(desc("pagerank")))


In [26]:
display(ranks.vertices)
display(ranks.edges)

DataFrame[id: string, pagerank: double]

DataFrame[src: string, dst: string, weight: double, weight: double]

In [29]:
# Search from "Esther" for users of age < 32.

ap1 = "5c5b3552b96c"
ap2 = "5c5b3552b5e3"
paths = g.bfs("id = ap1", "id = ap2")
paths.show()

# Specify edge filters or max path lengths.
g.bfs("id = ap1", "id = ap2",\
  edgeFilter="rssi > -75", maxPathLength=3)


AnalysisException: "cannot resolve '`ap1`' given input columns: [id]; line 1 pos 5;\n'Filter (id#463 = 'ap1)\n+- Deduplicate [id#463]\n   +- Project [ap#11 AS id#463]\n      +- Filter (site#10 = d1ee1d22-4b55-4c97-97c4-9d757144f45b)\n         +- Relation[site#10,ap#11,ap2#12,band#13,time_epoch#14,rssi#15] csv\n"

In [30]:
# g.connectedComponents()

result1 = g.stronglyConnectedComponents(maxIter=10)
result1.select("id", "component").orderBy("component").show()

+------------+------------+
|          id|   component|
+------------+------------+
|5c5b3552b796|274877906944|
|5c5b3552b714|274877906944|
|5c5b3552b660|274877906944|
|5c5b3552b70a|274877906944|
|5c5b3552b7cd|274877906944|
|5c5b3552b94e|274877906944|
|5c5b3552ba70|274877906944|
|5c5b3552b5e3|274877906944|
|5c5b3552bb6f|274877906944|
|5c5b3552b96c|274877906944|
|d420b000a866|558345748480|
+------------+------------+



In [31]:
result1.

DataFrame[id: string, component: bigint]

In [32]:
result2 = g.stronglyConnectedComponents(maxIter=10)
result2.select("id", "component").orderBy("component").show()

+------------+------------+
|          id|   component|
+------------+------------+
|5c5b3552b70a|274877906944|
|5c5b3552b796|274877906944|
|5c5b3552b7cd|274877906944|
|5c5b3552b660|274877906944|
|5c5b3552bb6f|274877906944|
|5c5b3552b714|274877906944|
|5c5b3552b96c|274877906944|
|5c5b3552b5e3|274877906944|
|5c5b3552b94e|274877906944|
|5c5b3552ba70|274877906944|
|d420b000a866|558345748480|
+------------+------------+



In [33]:
result2

DataFrame[id: string, component: bigint]

In [39]:
date_day + hr

'2020-03-1920'

In [49]:
s3_out_bucket = "s3://mist-test-bucket/" #top_1_time_epoch_by_site_ap_ap2_band/{}/{}/".format(date_day, hr)
s3_out_bucket_vertices = s3_out_bucket #+ "vertices/"
s3_out_bucket_edges = s3_out_bucket #+ "edges/"

print(s3_out_bucket_vertices, s3_out_bucket_edges)
g.vertices.write.parquet(s3_out_bucket_vertices)
g.edges.write.parquet(s3_out_bucket_vertices)
# s3_out_bucket


s3://mist-test-bucket/ s3://mist-test-bucket/


Py4JJavaError: An error occurred while calling o432.parquet.
: org.apache.hadoop.fs.s3a.AWSS3IOException: getFileStatus on : com.amazonaws.services.s3.model.AmazonS3Exception: The provided token has expired. (Service: Amazon S3; Status Code: 400; Error Code: ExpiredToken; Request ID: 94C7F957ACE921E9), S3 Extended Request ID: AkMigxMFtCX4gAWlM4ADF02zC8komIGrBGt27VdO3KxwIRd0y2pMzgQKayQDapblniWDzPJ9uEE=: The provided token has expired. (Service: Amazon S3; Status Code: 400; Error Code: ExpiredToken; Request ID: 94C7F957ACE921E9)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:178)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:1635)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:117)
	at org.apache.hadoop.fs.FileSystem.exists(FileSystem.java:1437)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.exists(S3AFileSystem.java:2040)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(Ins

In [41]:
# s3_out_bucket, rrm_local_path

In [30]:
g.persist()

['DST',
 'ID',
 'SRC',
 '_ATTR',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_edges',
 '_jvm_gf_api',
 '_jvm_graph',
 '_sc',
 '_sqlContext',
 '_vertices',
 'aggregateMessages',
 'bfs',
 'cache',
 'connectedComponents',
 'degrees',
 'dropIsolatedVertices',
 'edges',
 'filterEdges',
 'filterVertices',
 'find',
 'inDegrees',
 'labelPropagation',
 'outDegrees',
 'pageRank',
 'parallelPersonalizedPageRank',
 'persist',
 'pregel',
 'shortestPaths',
 'stronglyConnectedComponents',
 'svdPlusPlus',
 'triangleCount',
 'triplets',
 'unpersist',
 'vertices']

In [ ]:
# graph.degrees.filter("id = 1").show()

In [ ]:
GraphFrame provides the following built-in algorithms:
Connected components

Label propagation

PageRank

SVD++

Shortest Path

Strongly connected components

Triangle count